 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Fully-connected-linear-network" data-toc-modified-id="Fully-connected-linear-network-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Fully connected linear network</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Get-temperature-data" data-toc-modified-id="Get-temperature-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get temperature data</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Build-fully-connected-model" data-toc-modified-id="Build-fully-connected-model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Build fully connected model</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Predict-for-one-day" data-toc-modified-id="Predict-for-one-day-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Predict for one day</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Post-processing-with-rolling-window-for-2016" data-toc-modified-id="Post-processing-with-rolling-window-for-2016-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Post processing with rolling window for 2016</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Train-2015,-predict-2016" data-toc-modified-id="Train-2015,-predict-2016-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Train 2015, predict 2016</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Neural-network-with-one-hidden-layer" data-toc-modified-id="Neural-network-with-one-hidden-layer-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Neural network with one hidden layer</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Build-network" data-toc-modified-id="Build-network-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Build network</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Train-2015,-predict-2016" data-toc-modified-id="Train-2015,-predict-2016-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Train 2015, predict 2016</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Hidden-model-with-station-embeddings" data-toc-modified-id="Hidden-model-with-station-embeddings-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hidden model with station embeddings</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Build-embedding-model" data-toc-modified-id="Build-embedding-model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Build embedding model</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Train-2015,-predict-2016" data-toc-modified-id="Train-2015,-predict-2016-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Train 2015, predict 2016</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Embedding-size-hyper-parameter-tuning" data-toc-modified-id="Embedding-size-hyper-parameter-tuning-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Embedding size hyper-parameter tuning</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Adding-additional-variables-to-hidden-layer-model" data-toc-modified-id="Adding-additional-variables-to-hidden-layer-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Adding additional variables to hidden layer model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Get-additional-variables" data-toc-modified-id="Get-additional-variables-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Get additional variables</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Build-model-and-predict-for-2016" data-toc-modified-id="Build-model-and-predict-for-2016-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Build model and predict for 2016</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Hidden-nodes-hyper-parameter-tuning" data-toc-modified-id="Hidden-nodes-hyper-parameter-tuning-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Hidden nodes hyper-parameter tuning</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Additional-variables-with-the-embedding-model" data-toc-modified-id="Additional-variables-with-the-embedding-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Additional variables with the embedding model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Build-model-and-predict-for-2016" data-toc-modified-id="Build-model-and-predict-for-2016-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Build model and predict for 2016</a></span></li></ul></li></ul></div>

# Fully connected neural networks

In this notebook we will expand the simple EMOS linear network to fully connected non-linear neural nets. Furthermore, we will also use auxiliary data.

In [41]:
# Imports
from importlib import reload
import emos_network_theano; reload(emos_network_theano)
from  emos_network_theano import EMOS_Network
from crps_loss import crps_cost_function
import utils; reload(utils)
from utils import *
from datetime import datetime
%matplotlib inline
import matplotlib.pyplot as plt
from collections import OrderedDict
# import the keras modules
# Note that the cost function only works with the theano backend
import keras
from keras.layers import Input, Dense, merge, Embedding, Flatten, Dropout
from keras.layers.merge import Concatenate
from keras.models import Model
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

In [2]:
# Basic setup
DATA_DIR = '/Volumes/STICK/data/ppnn_data/'  # Mac
# DATA_DIR = '/project/meteo/w2w/C7/ppnn_data/'   # LMU
results_dir = '../results/'
window_size = 25   # Days in rolling window
fclt = 48   # Forecast lead time in hours

## Fully connected linear network

As a first step, we can build a linear model which also connects the means and standard deviations.

### Get temperature data

This follows the steps in the EMOS Network data

In [3]:
date_str = '2011-02-14'
train_set, test_set = get_train_test_sets(DATA_DIR, predict_date=date_str,
                                          fclt=fclt, window_size=window_size)

train set contains 25 days
test set contains 1 days


### Build fully connected model

In [16]:
def build_fc_model():
    inp = Input(shape=(2,))
    x = Dense(2, activation='linear')(inp)
    return Model(inputs=inp, outputs=x)

In [33]:
fc_model = build_fc_model()
fc_model.compile(optimizer=SGD(0.1), loss=crps_cost_function)

In [34]:
fc_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


Now we have 6 parameters instead of 4 with the standard EMOS Network.

### Predict for one day

In [35]:
# Define some parameters
early_stopping_delta = 1e-4   # How much the CRPS must improve before stopping
steps_max = 1000   # How many steps to fit at max
batch_size = train_set.features.shape[0]

In [36]:
fc_model.fit(train_set.features, train_set.targets, epochs=steps_max, 
             batch_size=batch_size,
             validation_data=[test_set.features, test_set.targets], 
             verbose=0,
             callbacks=[EarlyStopping(monitor='loss', 
                                      min_delta=early_stopping_delta,
                                      patience=2)])

In [37]:
# Get train and test CRPS
(fc_model.evaluate(train_set.features, train_set.targets, batch_size, verbose=0), 
 fc_model.evaluate(test_set.features, test_set.targets, batch_size, verbose=0))

(1.128275278515128, 0.76544715770375926)

For this particular day we get a score that is slightly better than the standard EMOS network.

### Post processing with rolling window for 2016

As with the EMOS models let's do a rolling window global post-processing for 2016.

In [38]:
date_str_start = '2016-01-01'
date_str_stop = '2017-01-01'

In [39]:
fc_model = build_fc_model()
fc_model.compile(optimizer=SGD(0.1), loss=crps_cost_function, 
                    metrics=[crps_cost_function])
# Note that we have to define a metric in the corrent loop implementation

In [42]:
# Use the loop function in utils
train_crps_list, valid_crps_list, results_df = loop_over_days(
    DATA_DIR,
    fc_model,
    date_str_start, date_str_stop, 
    window_size=window_size,
    fclt=fclt,     
    epochs_max=steps_max, 
    early_stopping_delta=early_stopping_delta, 
    lr=0.1,   
    verbose=0)

100%|██████████| 366/366 [07:54<00:00,  1.05s/it]


In [43]:
np.mean(train_crps_list), np.mean(valid_crps_list)

(0.98793579384560282, 1.0053749729681178)

So we get a slightly better training score and a slightly worse test score. This is a sign of overfitting. But the differences are small.

In [44]:
results_df.to_csv(results_dir + 'fc_network_rolling_window.csv')

### Train 2015, predict 2016

In [45]:
train_dates = ['2015-01-01', '2016-01-01']
test_dates =  ['2016-01-01', '2017-01-01']
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates, test_dates)

train set contains 365 days
test set contains 366 days


In [49]:
fc_model = build_fc_model()
fc_model.compile(optimizer=Adam(0.1), loss=crps_cost_function)

In [50]:
fc_model.fit(train_set.features, train_set.targets, epochs=10, batch_size=1024,
             validation_data=[test_set.features, test_set.targets])

Train on 180849 samples, validate on 182218 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 3.2375 - val_loss: 2.1826
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 1.4673 - val_loss: 1.0894
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 1.0780 - val_loss: 1.0150
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 1.0694 - val_loss: 1.0116
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 1.0692 - val_loss: 1.0126
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 1.0692 - val_loss: 1.0118
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 1.0692 - val_loss: 1.0121
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 1.0692 - val_loss: 1.0122
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 1.0693 - val_loss: 1.0124
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 1.0

Very similar to the standard EMOS Network. This indicates that there is not much additional information in the two extra connections we added.

In [1]:
preds = fc_model.predict(test_set.features)
results_df = create_results_df(test_set.date_strs, test_set.station_ids,
                               preds[:, 0], preds[:, 1])
results_df.to_csv(results_dir + 'fc_network_train_2015_pred_2016.csv')

NameError: name 'fc_model' is not defined

## Neural network with one hidden layer

Now we will build the first neural network with a hidden layer and a non-linear activation function. We will restrict ourselves to testing the 2015 training, 2016 prediction case.

### Build network

In [53]:
# Define parameters
hidden_nodes = 10

In [54]:
def build_hidden_model(hidden_nodes, feature_size=2):
    inp = Input(shape=(feature_size,))
    x = Dense(hidden_nodes, activation='relu')(inp)
    x = Dense(2, activation='linear')(x)
    return Model(inputs=inp, outputs=x)

In [55]:
hidden_model = build_hidden_model(hidden_nodes)
hidden_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 22        
Total params: 52
Trainable params: 52
Non-trainable params: 0
_________________________________________________________________


In [61]:
hidden_model.compile(optimizer=Adam(0.01), loss=crps_cost_function)

### Train 2015, predict 2016

In [63]:
# We can use the same data from above!
hidden_model.fit(train_set.features, train_set.targets, epochs=10, batch_size=1024,
                 validation_data=[test_set.features, test_set.targets])

Train on 180849 samples, validate on 182218 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 1.0705 - val_loss: 1.0128
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 1.0700 - val_loss: 1.0142
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 1.0709 - val_loss: 1.0144
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 1.0699 - val_loss: 1.0140
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 1.0705 - val_loss: 1.0116
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 1.0711 - val_loss: 1.0136
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 1.0703 - val_loss: 1.0152
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 1.0702 - val_loss: 1.0134
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 1.0701 - val_loss: 1.0129
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 1.0

The training loss is consistently smaller than the validation loss. This indicates overfitting. We could use Dropout for regularization. But since this model is just an intermediate step anyway, we will ignore that for now. The differences are small anyway.

In [64]:
preds = hidden_model.predict(test_set.features)
results_df = create_results_df(test_set.date_strs, test_set.station_ids,
                               preds[:, 0], preds[:, 1])
results_df.to_csv(results_dir + 'hidden_nn_train_2015_pred_2016.csv')

## Hidden model with station embeddings

Next we will add a station embedding

### Build embedding model

In [65]:
def build_emb_model(hidden_nodes, emb_size, max_id, feature_size=2):
    features_in = Input(shape=(feature_size,))
    id_in = Input(shape=(1,))
    emb = Embedding(max_id + 1, emb_size)(id_in)
    emb = Flatten()(emb)
    x = Concatenate()([features_in, emb])
    x = Dense(hidden_nodes, activation='relu')(x)
    x = Dense(2, activation='linear')(x)
    model = Model(inputs=[features_in, id_in], outputs=x)
    return model

In [67]:
emb_size = 5
max_id = int(np.max([train_set.cont_ids.max(), test_set.cont_ids.max()]))
hidden_nodes, max_id

(10, 536)

In [68]:
emb_model = build_emb_model(hidden_nodes, emb_size, max_id)
emb_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_11 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 5)          2685        input_11[0][0]                   
____________________________________________________________________________________________________
input_10 (InputLayer)            (None, 2)             0                                            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 5)             0           embedding_1[0][0]                
___________________________________________________________________________________________

In [69]:
emb_model.compile(optimizer=Adam(0.01), loss=crps_cost_function)

### Train 2015, predict 2016

In [70]:
emb_model.fit([train_set.features, train_set.cont_ids], train_set.targets, 
              epochs=10, batch_size=1024, 
              validation_data=[[test_set.features, test_set.cont_ids], test_set.targets])

Train on 180849 samples, validate on 182218 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 3.6400 - val_loss: 1.3108
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 1.0037 - val_loss: 0.9209
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 0.9754 - val_loss: 0.9171
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 0.9723 - val_loss: 0.9148
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 0.9724 - val_loss: 0.9157
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 0.9710 - val_loss: 0.9163
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 0.9713 - val_loss: 0.9192
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 0.9713 - val_loss: 0.9137
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 0.9708 - val_loss: 0.9141
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 0.9

In [72]:
preds = emb_model.predict([test_set.features, test_set.cont_ids])
results_df = create_results_df(test_set.date_strs, test_set.station_ids,
                               preds[:, 0], preds[:, 1])
results_df.to_csv(results_dir + 'embedding_nn_train_2015_pred_2016.csv')

### Embedding size hyper-parameter tuning

Since embeddings appear to work very well, we will test the impact of the embedding size before building more complex models. Of course, a larger embedding size might be useful when adding more variables, but this should give us some feeling.

In [73]:
def build_and_run_emb_model(emb_size):
    emb_model = build_emb_model(hidden_nodes, emb_size, max_id)
    emb_model.compile(optimizer=Adam(0.01), loss=crps_cost_function)
    emb_model.fit([train_set.features, train_set.cont_ids], train_set.targets, 
                  epochs=20,batch_size=1024, verbose=0,
                  validation_data=[[test_set.features, test_set.cont_ids], test_set.targets])
    print(emb_model.evaluate([train_set.features, train_set.cont_ids], train_set.targets, verbose=0),
          emb_model.evaluate([test_set.features, test_set.cont_ids], test_set.targets, verbose=0))

In [74]:
for emb_size in [1, 2, 3, 5, 10, 20]:
    print(emb_size)
    build_and_run_emb_model(emb_size)

1
0.968565424108 0.929144545723
2
0.960039174862 0.919980559779
3
0.966939299072 0.914936043469
5
0.957627968952 0.926167305927
10
0.956032127737 0.92702534406
20
0.968370541059 0.920356115977


Note that there is some variability. In our first experiment above with an embedding size of 5 we got a better score than here. For this very simple network an embedding size of two seems sufficient.

## Adding additional variables to hidden layer model

Now we can try adding additional variables.

### Get additional variables

Using the function defined in utils.

In [75]:
# The prepare_data function takes an ordered dict as an input
aux_dict = OrderedDict()
aux_dict['data_aux_geo_interpolated.nc'] = ['orog', 
                                            'station_alt', 
                                            'station_lat', 
                                            'station_lon']
aux_dict['data_aux_pl500_interpolated_00UTC.nc'] = ['u_pl500_fc',
                                                    'v_pl500_fc',
                                                    'gh_pl500_fc']
aux_dict['data_aux_pl850_interpolated_00UTC.nc'] = ['u_pl850_fc',
                                                    'v_pl850_fc',
                                                    'q_pl850_fc']
aux_dict['data_aux_surface_interpolated_00UTC.nc'] = ['cape_fc',
                                                      'sp_fc',
                                                      'tcc_fc']

In [76]:
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates, test_dates,
                                         aux_dict=aux_dict)

train set contains 365 days
test set contains 366 days


In [77]:
train_set.features.shape

(180849, 24)

### Build model and predict for 2016

In [85]:
hidden_model = build_hidden_model(hidden_nodes, feature_size=train_set.features.shape[1])
hidden_model.compile(optimizer=Adam(0.001), loss=crps_cost_function)

I found that I had to reduce the learning rate here. Otherwise I seem to get stuck in local minima.

In [86]:
hidden_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 24)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                250       
_________________________________________________________________
dense_28 (Dense)             (None, 2)                 22        
Total params: 272
Trainable params: 272
Non-trainable params: 0
_________________________________________________________________


In [91]:
# Note that I am running this cell multiple times
hidden_model.fit(train_set.features, train_set.targets, epochs=10, batch_size=1024,
                 validation_data=[test_set.features, test_set.targets])

Train on 180849 samples, validate on 182218 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 0.9508 - val_loss: 0.9414
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 0.9503 - val_loss: 0.9391
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 0.9500 - val_loss: 0.9395
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 0.9494 - val_loss: 0.9405
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 0.9490 - val_loss: 0.9404
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 0.9485 - val_loss: 0.9397
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 0.9482 - val_loss: 0.9403
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 0.9482 - val_loss: 0.9393
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 0.9474 - val_loss: 0.9408
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 0.9

So we see a definite improvement using auxiliary variables.

### Hidden nodes hyper-parameter tuning

Let's see what the impact of the number of hidden nodes is.

In [97]:
def build_and_run_hidden_model(hidden_nodes):
    hidden_model = build_hidden_model(hidden_nodes, feature_size=train_set.features.shape[1])
    hidden_model.compile(optimizer=Adam(0.001), loss=crps_cost_function)
    hidden_model.fit(train_set.features, train_set.targets, epochs=50, 
                  batch_size=1024, verbose=0,
                  validation_data=[test_set.features, test_set.targets])
    print(hidden_model.evaluate(train_set.features, train_set.targets, verbose=0),
          hidden_model.evaluate(test_set.features, test_set.targets, verbose=0))

In [98]:
for hidden_nodes in [10, 25, 50, 100, 250, 1000]:
    print(hidden_nodes)
    build_and_run_hidden_model(hidden_nodes)

10
0.966121331424 0.93475101895
25
0.942864599186 0.933350656875
50
0.934860278494 0.933259091651
100
0.905984635154 0.939907191921
250
0.879771149796 0.933663448304
1000
0.850411047073 0.952754239677


So with a larger network we get serious overfitting. Again, I would think that adding regularization with a larger network would maybe give the best results. Something to explore later.

In [99]:
# Let's make a submission with 50 hidden nodes
hidden_model = build_hidden_model(hidden_nodes=50, feature_size=train_set.features.shape[1])
hidden_model.compile(optimizer=Adam(0.001), loss=crps_cost_function)

In [103]:
# Note that I am running this cell multiple times
hidden_model.fit(train_set.features, train_set.targets, epochs=10, batch_size=1024,
                 validation_data=[test_set.features, test_set.targets])

Train on 180849 samples, validate on 182218 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 0.9500 - val_loss: 0.9382
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 0.9491 - val_loss: 0.9357
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 0.9468 - val_loss: 0.9364
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 0.9456 - val_loss: 0.9398
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 0.9441 - val_loss: 0.9357
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 0.9429 - val_loss: 0.9364
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 0.9418 - val_loss: 0.9372
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 0.9407 - val_loss: 0.9344
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 0.9396 - val_loss: 0.9364
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 0.9

In [104]:
preds = hidden_model.predict(test_set.features)
results_df = create_results_df(test_set.date_strs, test_set.station_ids,
                               preds[:, 0], preds[:, 1])
results_df.to_csv(results_dir + 'hidden_nn_aux_train_2015_pred_2016.csv')

## Additional variables with the embedding model

### Build model and predict for 2016

In [105]:
hidden_nodes, emb_size, feature_size = 50, 5, train_set.features.shape[1]

In [108]:
emb_model = build_emb_model(hidden_nodes, emb_size, max_id, feature_size)
emb_model.compile(optimizer=Adam(0.001), loss=crps_cost_function)
emb_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_41 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_9 (Embedding)          (None, 1, 5)          2685        input_41[0][0]                   
____________________________________________________________________________________________________
input_40 (InputLayer)            (None, 24)            0                                            
____________________________________________________________________________________________________
flatten_9 (Flatten)              (None, 5)             0           embedding_9[0][0]                
___________________________________________________________________________________________

In [112]:
# Again I am running this multiple times
emb_model.fit([train_set.features, train_set.cont_ids], train_set.targets, epochs=10, 
              batch_size=1024, 
              validation_data=[[test_set.features, test_set.cont_ids], test_set.targets])

Train on 180849 samples, validate on 182218 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 0.8485 - val_loss: 0.8541
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 0.8475 - val_loss: 0.8543
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 0.8459 - val_loss: 0.8559
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 0.8449 - val_loss: 0.8562
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 0.8439 - val_loss: 0.8573
Epoch 6/10
180849/180849 [==============================] - 1s - loss: 0.8434 - val_loss: 0.8571
Epoch 7/10
180849/180849 [==============================] - 1s - loss: 0.8418 - val_loss: 0.8540
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 0.8412 - val_loss: 0.8611
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 0.8404 - val_loss: 0.8557
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 0.8

In [113]:
preds = emb_model.predict([test_set.features, test_set.cont_ids])
results_df = create_results_df(test_set.date_strs, test_set.station_ids,
                               preds[:, 0], preds[:, 1])
results_df.to_csv(results_dir + 'embedding_nn_aux_train_2015_pred_2016.csv')